In [1]:
#-*- coding:utf-8 -*-
#'''
#'''
import os
import numpy as np
import cv2
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import imutils
from pixellink_model import create_pixellink_model
from pixellink_utils import *



import time
import json

import tensorflow as tf
from crnn_model import model

_IMAGE_HEIGHT = 32

Using TensorFlow backend.


In [2]:
r_mean = 123.
g_mean = 117.
b_mean = 104.
rgb_mean = [r_mean, g_mean, b_mean]


#img_path = './samples/TOP_29_121222.jpg'
save_weights = './weights/pixellink.h5'
model_pixel = create_pixellink_model(acf='relu')
model_pixel.load_weights(save_weights)



In [3]:
def _sparse_matrix_to_list(sparse_matrix, char_map_dict=None):
    indices = sparse_matrix.indices
    values = sparse_matrix.values
    dense_shape = sparse_matrix.dense_shape

    # the last index in sparse_matrix is ctc blanck note
    with open('./char_map/char_map.json', 'r') as f:
        char_map_dict = json.load(f)
        

    assert(isinstance(char_map_dict, dict) and 'char_map_dict is not a dict')    

    dense_matrix =  len(char_map_dict.keys()) * np.ones(dense_shape, dtype=np.int32)
    for i, indice in enumerate(indices):
        dense_matrix[indice[0], indice[1]] = values[i]
    string_list = []
    for row in dense_matrix:
        string = []
        for val in row:
            string.append(_int_to_string(val, char_map_dict))
        string_list.append(''.join(s for s in string if s != '*'))
    return string_list

def _int_to_string(value, char_map_dict=None):

    with open('./char_map/char_map.json', 'r') as f:
        char_map_dict = json.load(f)
            
    assert(isinstance(char_map_dict, dict) and 'char_map_dict is not a dict')
    
    for key in char_map_dict.keys():
        if char_map_dict[key] == int(value):
            return str(key)
        elif len(char_map_dict.keys()) == int(value):
            return "" 
    raise ValueError('char map dict not has {:d} value. convert index to char failed.'.format(value))




In [4]:
input_image = tf.placeholder(dtype=tf.float32, shape=[1, _IMAGE_HEIGHT, None, 3])

with open('./char_map/char_map.json', 'r') as f:
    char_map_dict = json.load(f)

# initialise the net model
crnn_net = model.CRNNCTCNetwork(phase='test',
                                hidden_num=256,
                                layers_num=2,
                                num_classes=len(char_map_dict.keys()) + 1)


with tf.variable_scope('CRNN_CTC', reuse=False):
    net_out = crnn_net.build_network(input_image)

input_sequence_length = tf.placeholder(tf.int32, shape=[1], name='input_sequence_length')

ctc_decoded, ct_log_prob = tf.nn.ctc_beam_search_decoder(net_out, input_sequence_length, merge_repeated=True)


# set checkpoint saver
saver = tf.train.Saver()
save_path = tf.train.latest_checkpoint('./bs_synth90k_model/')


Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [5]:
def crnn_ctc_predicts(image):
    
    with tf.Session() as sess:
        # restore all variables
        saver.restore(sess=sess, save_path=save_path)
        #saver.restore(sess=sess, save_path='./bs_synth90k_model/crnn_ctc_ocr_2018-11-16-03-14-19.ckpt-19999')


        #image = cv2.imread(image_test_path)
        h, w, c = image.shape
        height = _IMAGE_HEIGHT
        width = int(w * height / h)
        image = cv2.resize(image, (width, height))
        image = np.expand_dims(image, axis=0)
        image = np.array(image, dtype=np.float32)
        seq_len = np.array([width / 4], dtype=np.int32)

        preds = sess.run(ctc_decoded, feed_dict={input_image:image, input_sequence_length:seq_len})

        preds = _sparse_matrix_to_list(preds[0], char_map_dict)

        #print('Predict {:s} image as: {:s}'.format('1_AFTERSHAVE_1509', preds[0]))
    return preds

In [6]:
# 設定 影片 解析度
video_width = 640
video_height = 480

#cap = cv2.VideoCapture(0)   # device number 0
cap = cv2.VideoCapture(0)   # device number 1
cap.set(cv2.CAP_PROP_FRAME_WIDTH, video_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, video_height)
print("capture device is open: " + str(cap.isOpened()))

capture device is open: True


# While-loop 主程式

In [7]:
# loop to read image  
while True:

    # keyboard input value
    key = cv2.waitKey(1) & 0xFF

    # read from camera
    ret, frame = cap.read()

    if ret:  # success read
        image, *r = resize_image(frame)
        image_ori = image.copy()
        image = image[...,::-1] - rgb_mean
        image = np.expand_dims(image, axis=0)

        pixel_pos_scores, link_pos_scores = model_pixel.predict(image)
        pixel_pos_scores = softmax(pixel_pos_scores, axis=-1)
        link_pos_scores_reshaped = link_pos_scores.reshape(link_pos_scores.shape[:-1]+(8, 2))
        link_pos_scores = softmax(link_pos_scores_reshaped, axis=-1)

        #masks = decode_batch(pixel_pos_scores, link_pos_scores, pixel_conf_threshold=0.75, link_conf_threshold=0.9)
        masks = decode_batch(pixel_pos_scores, link_pos_scores, pixel_conf_threshold=0.75, link_conf_threshold=0.98)

        bboxes = mask_to_bboxes(masks[0], image_ori.shape, min_area=300, min_height=30)
        
        
        image_c = frame.copy()
        for box in bboxes:
            points = np.reshape(box, [4, 2])
            cv2.line(image_c,tuple(points[0]),tuple(points[1]),(0,0,255),2)
            cv2.line(image_c,tuple(points[0]),tuple(points[3]),(0,0,255),2)
            cv2.line(image_c,tuple(points[1]),tuple(points[2]),(0,0,255),2)
            cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
            
            # 裁切區域的 x 與 y 座標（左上角）
            x = int(box[2])
            y = int(box[5])

            # 裁切區域的長度與寬度
            h = int(box[1] - box[5])
            w = int(box[6] - box[2])

            # 裁切圖片
            if h >10 and w > 20 and x > 50 and y > 50:
                crop_img = image_c[y:y+h, x:x+w]

                predics = crnn_ctc_predicts(crop_img)

                predics_text = '{:s}'.format(predics[0])
                cv2.putText(frame, str(predics_text) , (x+5, y+5), cv2.FONT_HERSHEY_COMPLEX_SMALL,1, (255, 0, 255), 1, cv2.LINE_AA)
                #print('Predict image as: {:s}'.format(predics[0]))
            
        
    
        #cv2.putText(frame, str(frame_count) , (40, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL,1, (255, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('frame', frame)
        cv2.imshow('detect result', image_c)
        
    else:
        print('camera read fail.')
        break

    #if frame_count > FRAME_NUM:   # FPS: 30, so 30秒產生 900張
    #    break
    if key == ord('q'):     # press 'q' to leave while
        break
    #if key == ord(' '):     # press 's' to start
    #    print('Starting capture video...')
    #    start_video = True

# release VideoWriter object clsoe all windows
#out.release()
cap.release()
cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from ./bs_synth90k_model/crnn_ctc_ocr_2018-11-16-03-14-19.ckpt-19999


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

2 root error(s) found.
  (0) Not found: Key conv1_conv1_1/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at C:\Users\duchu\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
  (1) Not found: Key conv1_conv1_1/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at C:\Users\duchu\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
	 [[save/RestoreV2/_117]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "C:\Users\duchu\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\duchu\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\d